Steps
- [ ] meshing GEO > MSH > VTK > VTU
- [ ] repairing mesh (removing vertices)
- [ ] project creation PRJ
- [ ] geometries GML
- [ ] run simulation OGS
- [ ] postprocessing OGSTools

In [1]:
#1: meshing GEO > MSH > VTK
import gmsh
import sys

gmsh.initialize(sys.argv)
gmsh.model.add("square_with_inner_point")

# Mesh size
lc = 5
lp = 1

# 1. Define Geometry: 4 points for the outer square
p1 = gmsh.model.geo.addPoint(-50, -50, 0, lc)
p2 = gmsh.model.geo.addPoint( 50, -50, 0, lc)
p3 = gmsh.model.geo.addPoint( 50,  50, 0, lc)
p4 = gmsh.model.geo.addPoint(-50,  50, 0, lc)

# 2. Define the INNER POINT
inner_point_tag = gmsh.model.geo.addPoint(0, 0, 0, lp) # Point at origin

# 3. Create lines and surface
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

curve_loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
surface_tag = gmsh.model.geo.addPlaneSurface([curve_loop])

# SYNC is CRITICAL before embedding
gmsh.model.geo.synchronize()

# 4. EMBED the inner point into the surface
# This forces a mesh node at the exact coordinate
gmsh.model.mesh.embed(0, [inner_point_tag], 2, surface_tag)

# ===== 2. SET *ALL* MESH OPTIONS **BEFORE** GENERATING =====
# CRITICAL: Set options BEFORE gmsh.model.mesh.generate()
gmsh.option.setNumber("Mesh.SaveAll", 0)        # Primary setting
gmsh.option.setNumber("Mesh.SaveGroupsOfNodes", 0) # Also try setting this to 0
# ===========================================================

# 5. Generate 2D mesh and save
gmsh.model.mesh.generate(2)

# ===== 4. OPTIONAL: Check what Gmsh *thinks* it will save =====
# Get all elements in the model
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements()
print("Element types present after generation:", elementTypes)
# Type 15 = Point (Vertex), Type 1 = Line, Type 2 = Triangle
# If you see '15' here, vertices are still present in the mesh database.
# ================================================================

gmsh.write("square_with_center_node.msh")
gmsh.write("square_with_center_node.vtk")

# Uncomment to view the result
gmsh.fltk.run()
gmsh.finalize()

Element types present after generation: [ 1  2 15]


In [2]:
import pyvista as pv
# Load your VTK file
mesh = pv.read('square_with_center_node.vtk')
mesh.save('square_with_center_node.vtu')
# Get mesh information
print(f"Number of points: {mesh.n_points}")
print(f"Number of cells: {mesh.n_cells}")
#print(f"Cell types: {mesh.cell_types}")
# See the names of all data arrays in your file
print("Available point data arrays:", mesh.point_data.keys())
print("Available cell data arrays:", mesh.cell_data.keys())

Number of points: 939
Number of cells: 1881
Available point data arrays: []
Available cell data arrays: []


#2a: not reliable working, Filter cannot be activated
How to Remove Vertex Cells: Using ParaView (Recommended for most users)

This is the method suggested in the ParaView community for filtering cells by their properties
- Open your .vtu file in ParaView.
- Apply the Threshold filter.
- In the filter properties, set the Scalar to "Vertex Count" (this array is created by another filter). To get it, first apply the Count Cell Vertices filter to your data
- Set the threshold range to start from 2 (e.g., 2 to a large number like 1000). This excludes all cells with 1 vertex (the Vertex cells).
- Click Apply. You can now save the filtered mesh as a new .vtu file.

#2b: working
How to Remove Vertex Cells: Using ParaView (Recommended for most users)

This is the method suggested in the ParaView community for filtering cells by their properties
- Open your .vtu file in ParaView.
- Apply the filter: Extract Cells by Type
- Mark only Cell Types to keep
- Click Apply. You can now save the filtered mesh as a new .vtu file.

In [3]:
import pyvista as pv
# Load your VTU file
mesh = pv.read('pumping-well.vtu')
# Plot as wireframe
##mesh.plot(show_edges=True, color='white', line_width=1)
#plot in x-y plane
# Create a plotter and add the mesh
plotter = pv.Plotter() # Create a plotting object[citation:2]
plotter.add_mesh(
    mesh,
    show_edges=True,  # Makes cell edges visible for a wireframe-like view[citation:1]
    color='lightblue'
)
# CRITICAL: Set the camera to view the X-Y plane (top-down view)
##plotter.view_xy() # Looks along the -Z axis[citation:3]
# Show the plot
##plotter.show()

Actor (0x1d9bdda99c0)
  Center:                     (0.0, 0.0, 0.0)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -5.000E+01, 5.000E+01
  Y Bounds                    -5.000E+01, 5.000E+01
  Z Bounds                    0.000E+00, 0.000E+00
  User matrix:                Identity
  Has mapper:                 True

Property (0x1d9bdda9ea0)
  Ambient:                     0.0
  Ambient color:               Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  Edge opacity:                1.0
 

In [7]:
#3: project creation PRJ
import os
import ogstools as ot

#prj = ot.Project(output_file=out_dir / "lauwerier_new.prj")
prj = ot.Project(output_file="pumping-well-created.prj")
prj.geometry.add_geometry(filename="pumping-well.gml")
#prj.mesh.add_mesh(filename="h-testing.vtu")
prj.mesh.add_mesh(filename="pumping-well.vtu")
#processes
prj.processes.set_process(
    name="LiquidFlow",
    type="LIQUID_FLOW",
    integration_order="2",
    specific_body_force="0 0",
)
prj.processes.add_process_variable(
    process_variable="process_variable", process_variable_name="pressure"
)
prj.processes.add_secondary_variable(internal_name="darcy_velocity", output_name="velocity")
#medium properties
#medium0
prj.media.add_property(
    medium_id="0", phase_type="AqueousLiquid", name="density", type="Constant", value="1000",
)
prj.media.add_property(
    medium_id="0", phase_type="AqueousLiquid", name="viscosity", type="Constant", value="1e-3",
)
prj.media.add_property(
    medium_id="0", name="porosity", type="Constant", value="1.0",
)
prj.media.add_property(
    medium_id="0", name="permeability", type="Constant", value="1e-13",
)
prj.media.add_property(
    medium_id="0", name="reference_temperature", type="Constant", value="293.15",
)
prj.media.add_property(
    medium_id="0", name="storage", type="Constant", value="1e-6",
)
#parameters
prj.parameters.add_parameter(name="p0", type="Constant", value="0")
prj.parameters.add_parameter(name="p_dbc", type="Constant", value="0")
prj.parameters.add_parameter(name="p_nbc", type="Constant", value="1e-3")
prj.parameters.add_parameter(name="p_left", type="Constant", value="1e6")
prj.parameters.add_parameter(name="p_right", type="Constant", value="0.0")
prj.parameters.add_parameter(name="constant_porosity_parameter", type="Constant", value="1")
prj.parameters.add_parameter(name="kappa1", type="Constant", value="1e-3")
#initial and boundary conditions
prj.process_variables.set_ic(
    process_variable_name="pressure", components="1", order="1", initial_condition="p0",
)
prj.process_variables.add_bc(
    process_variable_name="pressure", components="1", geometrical_set="h-testing", geometry="well", type="Dirichlet", parameter="p_left",
)
#solver
prj.nonlinear_solvers.add_non_lin_solver(
    name="basic_picard", type="Picard", max_iter="10", linear_solver="general_linear_solver", abstol="1e-6",
)
prj.linear_solvers.add_lin_solver(
    name="general_linear_solver",
    kind="lis",
    solver_type="CG",
    precon_type="DIAGONAL",
    max_iteration_step="10000",
    error_tolerance="1e-20",
)
#time loop
prj.time_loop.add_process(
    process="LiquidFlow",
    nonlinear_solver_name="basic_picard",
    convergence_type="DeltaX",
    norm_type="NORM2",
    time_discretization="BackwardEuler",
)
prj.time_loop.set_stepping(
    process="LiquidFlow",
    type="FixedTimeStepping",
    t_initial="0.0",
    t_end="86400000 ",
    repeat="1",
    delta_t="864000",
)
#output
prj.time_loop.add_output(
    type="VTK",
    prefix="h-testing",
    repeat="1",
    each_steps="10",
    variables=["pressure", "velocity"],
)
#
prj.write_input()

#4: geometries GML
<?xml version="1.0" encoding="ISO-8859-1"?>
<?xml-stylesheet type="text/xsl" href="OpenGeoSysGLI.xsl"?>
<OpenGeoSysGLI xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ogs="http://www.opengeosys.org">
    <name>h-testing</name>
    <points>
        <point id="0" x="-50.0" y="-50.0" z="0" name="inflow"/>
        <point id="1" x="50.0" y="-50.0" z="0"/>
        <point id="2" x="50.0" y="50.0" z="0" name="outflow"/>
        <point id="3" x="-50.0"  y="50.0" z="0"/>
        <point id="4" x="0"  y="0" z="0" name="well"/>
    </points>
    <polylines>
        <polyline id="0" name="left">
            <pnt>0</pnt>
            <pnt>3</pnt>
        </polyline>		
        <polyline id="1" name="right">
            <pnt>1</pnt>
            <pnt>2</pnt>
        </polyline>		
        <polyline id="2" name="boundary">
            <pnt>0</pnt>
            <pnt>1</pnt>
            <pnt>2</pnt>
            <pnt>3</pnt>
        </polyline>	
    </polylines>
</OpenGeoSysGLI>

In [8]:
#5: run simulation OGS
import os
from pathlib import Path
prj_file = "pumping-well.prj"
out_dir = Path(os.environ.get("OGS_TESTRUNNER_OUT_DIR", "_out"))
out_dir.mkdir(parents=True, exist_ok=True)
model = ot.Project(input_file=prj_file, output_file=f"{out_dir}/modified.prj")
model.write_input()
model.run_model(logfile=f"{out_dir}/out.txt", args=f"-o {out_dir} -m . -s .")
#prj.run_model(logfile=f"{out_dir}/out.txt", args=f"-o {out_dir} -m . -s .")

Simulation: _out\modified.prj
Status: terminated with error code 1.
Last 10 line of the log:
info: Parsing the OGS commandline passed
info: This is OpenGeoSys-6 version 6.5.6-509-g8baf03ff. Log version: 2, Log level: info.
info: OGS starts on 2026-01-19 12:03:23+0100 in serial mode / Python embedded mode.
info: Eigen use 8 threads
info: Reading project file _out\modified.prj.
error: File '.\square_with_center_node_clean.vtu' does not exist.
critical: C:/Users/gitlab/builds/t1_LckkzC/0/ogs/ogs/Applications/ApplicationsLib/ProjectData.cpp:183 `anonymous-namespace'::readSingleMesh() 
error: Could not read mesh from 'C:\User\00-OGS\benchmarks\pumping-well\square_with_center_node_clean.vtu' file. No mesh added.
Could not construct OGSSimulation object



RuntimeError: OGS execution was not successful.

In [ ]:
#6: postprocessing OGSTools
import ogstools as ot
import matplotlib.pyplot as plt
import numpy as np

#ms = ot.MeshSeries("h-testing.pvd")
ms = ot.MeshSeries("_out/h-testing.pvd")
print(ms.timevalues)

def plot_sample(var: ot.variables.Scalar) -> None:
    fig, ax = plt.subplots(figsize=[6, 3], dpi=150)
    ax.set_xlabel(r"$x$ / m")
    ax.set_ylabel(var.get_label())
    for mesh, t in zip(ms, ms.timevalues, strict=True):
        line_mesh = mesh.sample_over_line([-50, 0, 0], [50, 0, 0])
        vals = line_mesh.sample(mesh)[var.data_name]
        #assert np.all(np.abs(vals) <= max_vals[var.data_name]), max(abs(vals))
        ax.plot(line_mesh.points[:, 0], var.transform(vals), label=f"{t=}", lw=1.5)
    ax.legend()
    ax.grid()
    fig.tight_layout()

plot_sample(ot.variables.Scalar("pressure", "Pa", "Pa"))